0) Imports:

In [1]:
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import FAISS
from langchain.callbacks import get_openai_callback

1) Configuration:

2. Load PDF file:

In [2]:
pdf_file = open("data/servicios_ebook.pdf", "rb")
pdf_reader = PdfReader(pdf_file)

In [3]:
text = ""
for page in pdf_reader.pages:
    text += page.extract_text()

In [4]:
# Check a part of the pdf extracted:
text[1000:2000]

'ebes tener millones de dudas, porque como te comentamos, también estuvimos en tu lugar\nhace muchos años.\nPor todo lo que te comentamos, también abrimos esta vía de consultoría, donde nos puedes\npreguntar de todo (o casi todo) ya que las cuestiones legales y migratorias por ley , solo lo puede\ndar un Inmigration Adviser . Nosotros contamos con uno, y puedes ver sus servicios de visas\npara Nueva Zelanda . Si tienes consultas migratorias para Australia, nos puedes escribir a\ncontacto@destinoceania.com .\n¿En qué te podemos ayudar? Te contamos brevemente nuestra historia, tal vez te ayude a\ndecidirte con quién de nosotros tener la consultoría.Menú\n11/2/23, 6:40 PM Consultoría – Destino Oceanía\nhttps://destinoceania.com/servicios/consultoria/ 2/4\nPatricio Toranza\nCo-fundador de Destino Oceanía, especialista de Nueva Zelanda\nPato pasó por varios trabajos en la construcción mediante los cuales consiguió su\nresidencia permanente en Nueva Zelanda para luego montarse su propia empr

3. Create chunks:

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
)

In [6]:
chunks = text_splitter.split_text(text)

In [7]:
print(f"Number of chunks: {len(chunks)}")
print(f"Example chunk[7]: {chunks[7]}")

Number of chunks: 53
Example chunk[7]: Nueva Zelanda puede abrirte el camino para desarrollar tus habilidades, conocer gente de todo el mundo e insertarse
en el incipiente mercado de trabajo de este país. Te asistimos para conseguir tu V isa de Estudiante y que puedas
viajar a Nueva Zelanda.
Gestión de visas de Visitante para Nueva Zelanda
Las visas de visitantes son para aquellos ciudadanos de países no miembros del  NZ V isa W aiver program ,
y permiten viajar a Nueva Zelanda con propósitos tales como vacaciones, visitar familiares, amigos (incluso estudiarMenú
Índice de contenidos11/2/23, 6:41 PM Servicio de gestión de visas para Nueva Zelanda
https://destinoceania.com/servicios/gestion-visas-nueva-zelanda/ 2/3hasta un máximo de 3 meses). Las visas de visitante permiten normalmente una estadía máxima de 9 meses en
Nueva Zelanda y son el puntapié inicial para descubrir los increíbles lugares de este país.
Si estás interesado en los servicios de gestión de visas para Nueva Zelanda, po

4. Create Embeddings:

In [8]:
# This is a free use model, but you can use your own model
# Lite Version:
'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'  # 471M
# Base Version:
'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'  # 1.11G

'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'

In [9]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

In [10]:
# Embedding example:

model = SentenceTransformer(
    'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
sentence_embeddings = model.encode("This is only an example sentence.")

print("Sentence embeddings len:"+str(len(sentence_embeddings)))

Sentence embeddings len:384


In [11]:
sentence_embeddings

array([ 3.48671041e-02,  2.07877666e-01,  5.55473119e-02,  2.26788729e-01,
        3.93917382e-01, -5.18002957e-02,  3.94082218e-01,  2.87398428e-01,
        2.26167277e-01, -9.68607217e-02,  3.42971057e-01, -1.10691385e-02,
       -2.50375062e-01, -2.94712023e-03,  1.57670602e-01, -1.82870254e-02,
        1.86952010e-01,  2.19318554e-01, -5.59591711e-01, -9.48017463e-02,
        4.48828548e-01,  2.70089973e-02, -3.79897207e-02, -2.21552879e-01,
       -1.32406250e-01, -4.23402667e-01,  1.18720941e-01, -8.44372213e-02,
        1.28614604e-01,  1.59072608e-01, -3.28535885e-01,  2.03147739e-01,
        1.38282835e-01,  4.26404215e-02,  1.66529156e-02,  8.18621293e-02,
        2.27468997e-01,  1.41169682e-01,  9.63715017e-02,  1.60501093e-01,
       -3.18667144e-02, -1.61084950e-01, -2.06193980e-03,  4.69921917e-01,
       -4.87578660e-01,  2.01607142e-02,  1.80015922e-01, -7.42736980e-02,
       -5.34520112e-02, -3.56206030e-01, -5.88137172e-02,  9.26874429e-02,
       -5.39254010e-01, -

In [12]:
#  Create embeddings for the entire text:

knowledge_base = FAISS.from_texts(chunks, embeddings)

In [13]:
# Generate a question for testing:
question = "¿Qué ofrece el servicio de consultoría?"
docs = knowledge_base.similarity_search(question, 3)

In [14]:
docs

[Document(page_content='telefónica, pero también en lugares como “The Warehouse” en Nueva Zelanda o “Sim Corner” \nen Australia. G astón recomienda averiguar qué compañía tiene la mejor cobertura en la zona en \nla cual pensás residir y comprar una simcard acorde.  \n \nEl servicio prepago es siempre más caro que un plan mensual. Si tenés la certeza de que te vas a \nquedar por un largo tiempo  en el país, te recomendamos contratar un plan mensual, siempre \nleyendo la letra chica, ya que muchas veces tienen cláusulas de cancelación con penalizaciones \nsi terminas el contrato antes de cierto plazo.  \n \nTe sugerimos escuchar el siguiente episodio del podcast en el que charlamos en profundidad \nsobre este tema:  \n \n \nPrimeros pasos en Australia y Nueva Zelanda  \n \n \n  LOS CAMINOS PARA MIGRAR A OCEANÍA  \ndestinoceania.com  \n16 \nA continuación les dejamos checklists de los primeros pasos para cada país:  \nAUSTRALIA  \n• Arribo a aeropuertos internacionales : Sydney / Melbourn

5. Question to the PDF document using openapi:

In [15]:
# Loas openai api key
from dotenv import load_dotenv
import openai
import os

# load environment variables:
load_dotenv()

True

In [16]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain

In [17]:
# Load language model:
chat_model = ChatOpenAI(model_name='gpt-3.5-turbo')

chain = load_qa_chain(chat_model, chain_type="stuff")

In [18]:
new_question = """Cuando es el cumpleaños de maradona?"""
# Search similar phrases in the knowledge base:
new_docs = knowledge_base.similarity_search(new_question, 3)
# Use the similar phrases to generate the context to ChatOpenAI:
new_answer = chain.run(input_documents=new_docs, question=new_question)
print(f"ChatGPT Answer: {new_answer}")

ChatGPT Answer: No tengo información sobre la fecha de cumpleaños de Maradona.


6. Review the cost:

In [19]:
with get_openai_callback() as cb:
    response = chain.run(input_documents=new_docs, question=new_question)
    print(cb)

Tokens Used: 885
	Prompt Tokens: 876
	Completion Tokens: 9
Successful Requests: 1
Total Cost (USD): $0.001332
